In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
# !wget https://f000.backblazeb2.com/file/malaya-model/v34/stem/model.pb

In [5]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops
from glob import glob
tf.set_random_seed(0)

In [6]:
pbs = glob('*.pb')
pbs

['pretrained-speakernet.pb',
 'pretrained-vggvox-v1.pb',
 'pretrained-vggvox-v2.pb',
 'pretrained-deep-speaker.pb',
 'model.pb']

In [7]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in ['model.pb']:
    input_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder'],
                                           ['decode_1/greedy', 'decode_2/beam'], transforms)
    
    with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [ ]:
!rm *.pb*

In [ ]:
# converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()